In [1]:
import os
import urllib.request
from datetime import datetime
import pandas as pd
import re
from io import StringIO
import csv

In [8]:
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

province_ids = range(1, 28)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for pid in province_ids:
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={pid}&year1=1981&year2=2024&type=Mean"
    filename = os.path.join(DATA_DIR, f"province_{pid}_{timestamp}.csv")

    try:
        urllib.request.urlretrieve(url, filename)
        print(f"[OK] Завантажено: {filename}")
    except Exception as e:
        print(f"[Помилка] Область {pid}: {e}")

[OK] Завантажено: data\province_1_20250930_141409.csv
[OK] Завантажено: data\province_2_20250930_141409.csv
[OK] Завантажено: data\province_3_20250930_141409.csv
[OK] Завантажено: data\province_4_20250930_141409.csv
[OK] Завантажено: data\province_5_20250930_141409.csv
[OK] Завантажено: data\province_6_20250930_141409.csv
[OK] Завантажено: data\province_7_20250930_141409.csv
[OK] Завантажено: data\province_8_20250930_141409.csv
[OK] Завантажено: data\province_9_20250930_141409.csv
[OK] Завантажено: data\province_10_20250930_141409.csv
[OK] Завантажено: data\province_11_20250930_141409.csv
[OK] Завантажено: data\province_12_20250930_141409.csv
[OK] Завантажено: data\province_13_20250930_141409.csv
[OK] Завантажено: data\province_14_20250930_141409.csv
[OK] Завантажено: data\province_15_20250930_141409.csv
[OK] Завантажено: data\province_16_20250930_141409.csv
[OK] Завантажено: data\province_17_20250930_141409.csv
[OK] Завантажено: data\province_18_20250930_141409.csv
[OK] Завантажено: d

In [9]:
DATA_DIR = "data"
all_data = []

for file in sorted(os.listdir(DATA_DIR)):
    if not file.endswith(".csv"):
        continue
    filepath = os.path.join(DATA_DIR, file)

    with open(filepath, "r", encoding="utf-8", errors="replace") as f:
        header_idx = None
        for i, line in enumerate(f):
            if re.search(r"\byear\b", line, re.IGNORECASE) and re.search(r"\bweek\b", line, re.IGNORECASE):
                header_idx = i
                break
    if header_idx is None:
        continue  

    df = pd.read_csv(
        filepath,
        encoding="utf-8",
        skiprows=header_idx,
        dtype=str,           
        skip_blank_lines=True)

    df.columns = df.columns.str.strip().str.lower()

    expected_cols = ["year", "week", "smn", "smt", "vci", "tci", "vhi"]
    keep_cols = [c for c in expected_cols if c in df.columns]
    if not keep_cols:
        continue
    df = df[keep_cols].copy()

    df[keep_cols] = df[keep_cols].apply(pd.to_numeric, errors="coerce")

    m = re.search(r"province_(\d+)_", file)
    df["province_id"] = int(m.group(1)) if m else None

    all_data.append(df)

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
else:
    final_df = pd.DataFrame(columns=expected_cols + ["province_id"])

final_df = final_df.sort_values(["province_id", "year", "week"]
    ).reset_index(drop=True)

print("Унікальних province_id:", final_df["province_id"].nunique())
print("Список областей:", sorted(final_df["province_id"].unique()))

Унікальних province_id: 27
Список областей: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27)]


In [10]:
province_names = {
    1: "Вінницька",
    2: "Волинська",
    3: "Дніпропетровська",
    4: "Донецька",
    5: "Житомирська",
    6: "Закарпатська",
    7: "Запорізька",
    8: "Івано-Франківська",
    9: "Київська",
    10: "Кіровоградська",
    11: "Луганська",
    12: "Львівська",
    13: "Миколаївська",
    14: "Одеська",
    15: "Полтавська",
    16: "Рівненська",
    17: "Сумська",
    18: "Тернопільська",
    19: "Харківська",
    20: "Херсонська",
    21: "Хмельницька",
    22: "Черкаська",
    23: "Чернівецька",
    24: "Чернігівська",
    25: "Республіка Крим",
    26: "м. Київ",
    27: "Севастополь"
}

final_df["province_name"] = final_df["province_id"].map(province_names)

final_df = final_df[final_df!=-1]
final_df = final_df.dropna()
print(final_df.head(70000))

       year   week     smn    smt    vci    tci  province_id province_name
2       1.0  0.016  254.68   1.91  59.48  30.69            1     Вінницька
3       1.0  0.026  244.98  14.39  92.72  53.55            1     Вінницька
4       1.0  0.030  251.40  18.27  71.45  44.86            1     Вінницька
5       1.0  0.031  257.00  20.04  51.54  35.79            1     Вінницька
6       1.0  0.035  260.76  24.75  38.17  31.46            1     Вінницька
...     ...    ...     ...    ...    ...    ...          ...           ...
60393  52.0  0.108  259.79  64.38  50.75  57.57           27   Севастополь
60394  52.0  0.122  270.70  78.63   3.60  41.12           27   Севастополь
60395  52.0  0.128  269.82  84.38   7.56  45.97           27   Севастополь
60396  52.0  0.130  271.31  89.06   1.70  45.38           27   Севастополь
60397  52.0  0.143  268.13  75.65  15.22  45.44           27   Севастополь

[59022 rows x 8 columns]


In [11]:
def get_vci_series_by_province_and_year(df, province_id, year):
    
    print(f"--- Завдання: Ряд VCI для області {province_id} за {year} рік ---")

    # Перевірка наявності даних
    if 'vci' not in df.columns:
        print("Помилка: Стовпець 'vci' не знайдено у DataFrame. Використовуйте 'tci' або інший доступний показник.")
        return pd.Series()

    # Фільтрація
    filtered_df = df[
        (df['province_id'] == province_id) &
        (df['year'] == year)
    ].sort_values(by='week')

    if filtered_df.empty:
        print(f"Дані для області {province_id} за {year} рік не знайдені.")
        return pd.Series()

    # Формування ряду
    vci_series = filtered_df.set_index('week')['vci']

    # Читабельний вивід
    province_name = filtered_df['province_name'].iloc[0] if 'province_name' in filtered_df.columns else f"ID: {province_id}"
    print(f"\nРяд VCI по тижнях для: {province_name}, {year} рік")
    print("--------------------------------------------------")
    print(vci_series.to_string())
    print("--------------------------------------------------")

    return vci_series

In [12]:
# Замініть ID та Рік на реальні значення у вашому датасеті
province_id_test = 1 # Вінницька
year_test = 2    # Ваш наявний рік
vci_series = get_vci_series_by_province_and_year(final_df, province_id_test, year_test)

--- Завдання: Ряд VCI для області 1 за 2 рік ---

Ряд VCI по тижнях для: Вінницька, 2 рік
--------------------------------------------------
week
0.013    61.99
0.026    90.50
0.029    74.33
0.032    53.81
0.032    82.61
0.036    41.42
0.036    52.62
0.036    57.06
0.041    36.91
0.041    32.68
0.042    76.06
0.044    49.76
0.045    58.32
0.046    60.86
0.047    52.12
0.047    22.66
0.049    29.70
0.051    46.90
0.052    48.33
0.053    49.37
0.054    31.75
0.054    28.41
0.054    46.76
0.054    33.60
0.055    49.96
0.056    54.25
0.058    14.31
0.060    45.77
0.060    48.16
0.061    35.64
0.063    40.84
0.067    16.10
0.067    22.76
0.068    35.99
0.068    32.07
0.076    14.32
0.080    18.72
0.084    22.75
0.090    20.80
0.101     5.89
--------------------------------------------------


In [13]:
def get_vci_series_by_provinces_and_years(df, province_ids, start_year, end_year):
    
    print(f"--- Завдання: Ряд VCI для областей {province_ids} за {start_year}-{end_year} роки ---")

    if 'vci' not in df.columns:
        print("Помилка: Стовпець 'vci' не знайдено у DataFrame.")
        return pd.DataFrame()

    # Фільтрація
    filtered_df = df[
        (df['province_id'].isin(province_ids)) &
        (df['year'] >= start_year) &
        (df['year'] <= end_year)
    ].sort_values(by=['province_id', 'year', 'week'])

    if filtered_df.empty:
        print(f"Дані для областей {province_ids} за {start_year}-{end_year} роки не знайдені.")
        return pd.DataFrame()

    # Вибірка необхідних стовпців
    result_df = filtered_df[['province_name', 'year', 'week', 'vci']]

    # Читабельний вивід
    province_names = result_df['province_name'].unique()
    print(f"\nВибірка VCI для областей: {', '.join(province_names)} за {start_year}-{end_year} роки ({len(result_df)} рядків)")
    print("--------------------------------------------------------------------------------")
    print(result_df.head(10).to_string()) # Вивід перших 10 рядків для компактності
    print("...")
    print(result_df.tail(10).to_string()) # Вивід останніх 10 рядків
    print("--------------------------------------------------------------------------------")

    return result_df

In [14]:
province_ids_test = [1, 2] # Вінницька та Волинська
start_year_test = 3
end_year_test = 5
vci_data_range = get_vci_series_by_provinces_and_years(final_df, province_ids_test, start_year_test, end_year_test)

--- Завдання: Ряд VCI для областей [1, 2] за 3-5 роки ---

Вибірка VCI для областей: Вінницька, Волинська за 3-5 роки (246 рядків)
--------------------------------------------------------------------------------
   province_name  year   week    vci
89     Вінницька   3.0  0.011  62.64
90     Вінницька   3.0  0.026  81.96
91     Вінницька   3.0  0.028  88.08
92     Вінницька   3.0  0.029  75.78
93     Вінницька   3.0  0.031  52.47
94     Вінницька   3.0  0.037  43.83
95     Вінницька   3.0  0.037  51.05
96     Вінницька   3.0  0.038  56.68
97     Вінницька   3.0  0.042  37.82
98     Вінницька   3.0  0.044  57.52
...
     province_name  year   week    vci
2442     Волинська   5.0  0.082  41.07
2443     Волинська   5.0  0.082  37.65
2444     Волинська   5.0  0.083  28.18
2445     Волинська   5.0  0.084  52.49
2446     Волинська   5.0  0.085  37.70
2447     Волинська   5.0  0.086  37.58
2448     Волинська   5.0  0.090  24.77
2449     Волинська   5.0  0.095   1.98
2450     Волинська   5.0  

In [15]:
def get_vci_statistics(df, province_ids, years):
   
    print(f"--- Завдання: Статистика VCI для областей {province_ids} за роки {years} ---")

    if 'vci' not in df.columns:
        print("Помилка: Стовпець 'vci' не знайдено у DataFrame.")
        return pd.DataFrame()

    # Фільтрація
    filtered_df = df[
        (df['province_id'].isin(province_ids)) &
        (df['year'].isin(years))
    ]

    if filtered_df.empty:
        print(f"Дані для областей {province_ids} за роки {years} не знайдені.")
        return pd.DataFrame()

    # Групування та агрегація
    stats_df = filtered_df.groupby(['province_id', 'province_name']).agg(
        vci_min=('vci', 'min'),
        vci_max=('vci', 'max'),
        vci_mean=('vci', 'mean'),
        vci_median=('vci', 'median'),
        total_records=('vci', 'count')
    ).reset_index()

    # Округлення
    for col in ['vci_min', 'vci_max', 'vci_mean', 'vci_median']:
        stats_df[col] = stats_df[col].round(3)

    # Читабельний вивід
    print(f"\nСтатистика VCI для {len(province_ids)} областей за {len(years)} років:")
    print("--------------------------------------------------")
    print(stats_df.to_string(index=False))
    print("--------------------------------------------------")

    return stats_df

In [16]:
province_ids_stats = [3, 4, 27] # Дніпропетровська, Донецька, Севастополь
years_stats = [8, 10]
vci_stats = get_vci_statistics(final_df, province_ids_stats, years_stats)

--- Завдання: Статистика VCI для областей [3, 4, 27] за роки [8, 10] ---

Статистика VCI для 3 областей за 2 років:
--------------------------------------------------
 province_id    province_name  vci_min  vci_max  vci_mean  vci_median  total_records
           3 Дніпропетровська     0.25    85.45    43.600       37.58             85
           4         Донецька     3.35    96.30    29.350       26.27             85
          27      Севастополь     0.54    91.80    42.442       39.81             85
--------------------------------------------------
